# 🎲 Monte Carlo en Sistemas Estocásticos: De Dados a Modelos Físicos

**Asignatura:** Física Estadística  
**Fecha:** Febrero 2026  

---

## 📋 Índice

1. **Ejercicio Base:** Probabilidades con 3 dados
2. **Visualizaciones Avanzadas:** Animaciones y comparaciones
3. **Extensiones Creativas:** Dados cargados, Craps, correlaciones
4. **Conceptos Avanzados:** Metropolis, muestreo de importancia
5. **Conexión con Física:** Random walk, difusión, modelos de espines

---

In [ ]:
# Setup inicial
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
from scipy import stats
from collections import Counter
import seaborn as sns

# Configuración de gráficas
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# Semilla aleatoria para reproducibilidad
np.random.seed(42)

print("✅ Setup completo")

---

# 1️⃣ SECCIÓN 1: Ejercicio Base

## Enunciado

Lanzamos **tres dados** de 6 caras. Calcular:

- **a)** P(suma ≤ 6)
- **b)** P(suma ≤ 16)
- **c)** P(7 ≤ suma ≤ 16)

---

## 1.1 Solución Analítica (Teoría de Probabilidad)

### Espacio de Resultados

- Número total de resultados posibles: $6^3 = 216$
- Suma mínima: 3 (1+1+1)
- Suma máxima: 18 (6+6+6)

### Método: Función Generatriz

La probabilidad de cada suma se puede calcular con la función generatriz:

$$G(x) = \left(\frac{x + x^2 + x^3 + x^4 + x^5 + x^6}{6}\right)^3$$

El coeficiente de $x^k$ nos da $P(\text{suma} = k)$.

In [ ]:
def calcular_probabilidades_analiticas():
    """
    Calcula la distribución de probabilidad exacta para la suma de 3 dados
    usando fuerza bruta (enumeración completa)
    """
    
    # Generar todas las combinaciones posibles (6^3 = 216)
    resultados = {}
    
    for d1 in range(1, 7):
        for d2 in range(1, 7):
            for d3 in range(1, 7):
                suma = d1 + d2 + d3
                resultados[suma] = resultados.get(suma, 0) + 1
    
    # Convertir a probabilidades
    total = 6**3
    probabilidades = {k: v/total for k, v in resultados.items()}
    
    return probabilidades


# Calcular distribución exacta
prob_exactas = calcular_probabilidades_analiticas()

# Mostrar tabla
print("📊 DISTRIBUCIÓN DE PROBABILIDAD EXACTA\n")
print(f"{'Suma':<6} {'Frecuencia':<12} {'Probabilidad':<15} {'Acumulada':<15}")
print("-" * 55)

acumulada = 0
for suma in sorted(prob_exactas.keys()):
    acumulada += prob_exactas[suma]
    print(f"{suma:<6} {int(prob_exactas[suma]*216):<12} {prob_exactas[suma]:.6f}      {acumulada:.6f}")


# Respuestas al ejercicio
print("\n" + "="*60)
print("RESPUESTAS (MÉTODO ANALÍTICO):")
print("="*60)

# a) P(suma ≤ 6)
prob_a = sum(prob_exactas[k] for k in prob_exactas if k <= 6)
print(f"\na) P(suma ≤ 6) = {prob_a:.6f} ({prob_a*100:.4f}%)")
print(f"   Casos favorables: {int(prob_a * 216)} de 216")

# b) P(suma ≤ 16)
prob_b = sum(prob_exactas[k] for k in prob_exactas if k <= 16)
print(f"\nb) P(suma ≤ 16) = {prob_b:.6f} ({prob_b*100:.4f}%)")
print(f"   Casos favorables: {int(prob_b * 216)} de 216")

# c) P(7 ≤ suma ≤ 16)
prob_c = sum(prob_exactas[k] for k in prob_exactas if 7 <= k <= 16)
print(f"\nc) P(7 ≤ suma ≤ 16) = {prob_c:.6f} ({prob_c*100:.4f}%)")
print(f"   Casos favorables: {int(prob_c * 216)} de 216")
print("\n" + "="*60)

---

## 1.2 Solución con Monte Carlo

### Principio del Método

En lugar de enumerar todos los casos, **simulamos** muchas tiradas aleatorias.

**Ley de los Grandes Números:**  
Si repetimos el experimento $N$ veces, la frecuencia relativa converge a la probabilidad real:

$$P(\text{evento}) \approx \frac{\text{# veces que ocurre}}{N}$$

In [ ]:
def simulacion_monte_carlo_dados(n_tiradas=1_000_000):
    """
    Simula N tiradas de 3 dados y calcula las probabilidades
    
    Args:
        n_tiradas: Número de simulaciones (mayor = más precisión)
    
    Returns:
        dict: Resultados de la simulación
    """
    
    # Simular todas las tiradas de una vez (vectorizado, más rápido)
    dados = np.random.randint(1, 7, size=(n_tiradas, 3))
    sumas = dados.sum(axis=1)
    
    # Calcular probabilidades por frecuencia
    contador = Counter(sumas)
    prob_simuladas = {k: v/n_tiradas for k, v in contador.items()}
    
    # Respuestas
    prob_a = np.sum(sumas <= 6) / n_tiradas
    prob_b = np.sum(sumas <= 16) / n_tiradas
    prob_c = np.sum((sumas >= 7) & (sumas <= 16)) / n_tiradas
    
    return {
        'sumas': sumas,
        'probabilidades': prob_simuladas,
        'prob_a': prob_a,
        'prob_b': prob_b,
        'prob_c': prob_c
    }


# Ejecutar simulación
print("🎲 Ejecutando simulación Monte Carlo (1,000,000 tiradas)...\n")
resultado_mc = simulacion_monte_carlo_dados(n_tiradas=1_000_000)

print("="*60)
print("RESPUESTAS (MÉTODO MONTE CARLO):")
print("="*60)

print(f"\na) P(suma ≤ 6) = {resultado_mc['prob_a']:.6f}")
print(f"b) P(suma ≤ 16) = {resultado_mc['prob_b']:.6f}")
print(f"c) P(7 ≤ suma ≤ 16) = {resultado_mc['prob_c']:.6f}")

# Comparación con solución analítica
print("\n" + "="*60)
print("COMPARACIÓN: ANALÍTICO vs MONTE CARLO")
print("="*60)

print(f"\n{'Pregunta':<10} {'Analítico':<15} {'Monte Carlo':<15} {'Error':<15}")
print("-" * 60)

error_a = abs(prob_a - resultado_mc['prob_a'])
error_b = abs(prob_b - resultado_mc['prob_b'])
error_c = abs(prob_c - resultado_mc['prob_c'])

print(f"{'a)':<10} {prob_a:.6f}      {resultado_mc['prob_a']:.6f}      {error_a:.6f}")
print(f"{'b)':<10} {prob_b:.6f}      {resultado_mc['prob_b']:.6f}      {error_b:.6f}")
print(f"{'c)':<10} {prob_c:.6f}      {resultado_mc['prob_c']:.6f}      {error_c:.6f}")

print("\n✅ El error es típicamente < 0.001 (0.1%) con 1M de tiradas")

---

## 1.3 Visualización: Comparación de Distribuciones

In [ ]:
# Comparación visual
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Gráfica 1: Histograma comparativo
sumas_posibles = sorted(prob_exactas.keys())
prob_exactas_vals = [prob_exactas[s] for s in sumas_posibles]
prob_mc_vals = [resultado_mc['probabilidades'].get(s, 0) for s in sumas_posibles]

x = np.arange(len(sumas_posibles))
width = 0.35

ax1.bar(x - width/2, prob_exactas_vals, width, label='Analítico', alpha=0.8, color='steelblue')
ax1.bar(x + width/2, prob_mc_vals, width, label='Monte Carlo', alpha=0.8, color='coral')

ax1.set_xlabel('Suma de los 3 dados', fontsize=12)
ax1.set_ylabel('Probabilidad', fontsize=12)
ax1.set_title('Distribución de Probabilidad: Analítico vs Monte Carlo', fontsize=14, fontweight='bold')
ax1.set_xticks(x)
ax1.set_xticklabels(sumas_posibles)
ax1.legend(fontsize=11)
ax1.grid(alpha=0.3)

# Gráfica 2: Distribución acumulada
prob_acum_exacta = []
prob_acum_mc = []
acum = 0
acum_mc = 0

for s in sumas_posibles:
    acum += prob_exactas[s]
    acum_mc += resultado_mc['probabilidades'].get(s, 0)
    prob_acum_exacta.append(acum)
    prob_acum_mc.append(acum_mc)

ax2.plot(sumas_posibles, prob_acum_exacta, 'o-', linewidth=2, markersize=8, 
         label='Analítico', color='steelblue')
ax2.plot(sumas_posibles, prob_acum_mc, 's--', linewidth=2, markersize=6, 
         label='Monte Carlo', color='coral', alpha=0.7)

# Marcar las respuestas a), b), c)
ax2.axhline(prob_a, color='red', linestyle=':', linewidth=2, alpha=0.5)
ax2.axvline(6, color='red', linestyle=':', linewidth=2, alpha=0.5, label='a) suma ≤ 6')

ax2.axhline(prob_b, color='green', linestyle=':', linewidth=2, alpha=0.5)
ax2.axvline(16, color='green', linestyle=':', linewidth=2, alpha=0.5, label='b) suma ≤ 16')

ax2.set_xlabel('Suma de los 3 dados', fontsize=12)
ax2.set_ylabel('Probabilidad Acumulada', fontsize=12)
ax2.set_title('Función de Distribución Acumulada (CDF)', fontsize=14, fontweight='bold')
ax2.legend(fontsize=10)
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("📊 Las distribuciones son prácticamente idénticas → Monte Carlo funciona")

---

## 1.4 Análisis de Convergencia

**Pregunta clave:** ¿Cuántas tiradas necesitamos para una buena aproximación?

**Teorema del Límite Central:** El error estándar escala como $1/\sqrt{N}$

In [ ]:
def analizar_convergencia():
    """
    Estudia cómo converge la simulación con el número de tiradas
    """
    
    n_valores = [100, 500, 1000, 5000, 10000, 50000, 100000, 500000, 1000000]
    errores_a = []
    errores_b = []
    errores_c = []
    
    for n in n_valores:
        res = simulacion_monte_carlo_dados(n_tiradas=n)
        
        errores_a.append(abs(prob_a - res['prob_a']))
        errores_b.append(abs(prob_b - res['prob_b']))
        errores_c.append(abs(prob_c - res['prob_c']))
    
    return n_valores, errores_a, errores_b, errores_c


print("🔬 Analizando convergencia (puede tardar ~30 segundos)...\n")
n_vals, err_a, err_b, err_c = analizar_convergencia()

# Gráfica log-log
plt.figure(figsize=(12, 6))
plt.loglog(n_vals, err_a, 'o-', linewidth=2, markersize=8, label='Error a)', color='red')
plt.loglog(n_vals, err_b, 's-', linewidth=2, markersize=8, label='Error b)', color='green')
plt.loglog(n_vals, err_c, '^-', linewidth=2, markersize=8, label='Error c)', color='blue')

# Línea teórica 1/√N
n_teorico = np.array(n_vals)
error_teorico = 0.01 / np.sqrt(n_teorico / 1000)
plt.loglog(n_teorico, error_teorico, '--', linewidth=2, color='black', 
          label='Teórico: 1/√N', alpha=0.7)

plt.xlabel('Número de tiradas (N)', fontsize=12)
plt.ylabel('Error absoluto', fontsize=12)
plt.title('Convergencia del Método Monte Carlo', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(alpha=0.3, which='both')
plt.tight_layout()
plt.show()

print("\n📈 CONCLUSIÓN:")
print("  - Con 1,000 tiradas → error ~1%")
print("  - Con 10,000 tiradas → error ~0.3%")
print("  - Con 1,000,000 tiradas → error ~0.03%")
print("\n  El error disminuye como 1/√N (pendiente -0.5 en log-log)")

---

# 2️⃣ SECCIÓN 2: Visualizaciones Avanzadas

## 2.1 Animación de Tiradas de Dados

Vamos a **visualizar los dados** físicamente mientras se tiran.

In [ ]:
def dibujar_dado(ax, valor, posicion, color='white'):
    """
    Dibuja un dado con el valor especificado
    """
    x, y = posicion
    
    # Cuadrado del dado
    square = plt.Rectangle((x, y), 1, 1, fill=True, facecolor=color, 
                          edgecolor='black', linewidth=3)
    ax.add_patch(square)
    
    # Puntos según el valor
    puntos_config = {
        1: [(0.5, 0.5)],
        2: [(0.3, 0.3), (0.7, 0.7)],
        3: [(0.3, 0.3), (0.5, 0.5), (0.7, 0.7)],
        4: [(0.3, 0.3), (0.3, 0.7), (0.7, 0.3), (0.7, 0.7)],
        5: [(0.3, 0.3), (0.3, 0.7), (0.5, 0.5), (0.7, 0.3), (0.7, 0.7)],
        6: [(0.3, 0.3), (0.3, 0.5), (0.3, 0.7), (0.7, 0.3), (0.7, 0.5), (0.7, 0.7)]
    }
    
    for px, py in puntos_config[valor]:
        ax.plot(x + px, y + py, 'ko', markersize=15)


def visualizar_tirada_simple():
    """
    Muestra una tirada de 3 dados con visualización
    """
    # Tirar dados
    dados = np.random.randint(1, 7, size=3)
    
    fig, ax = plt.subplots(figsize=(10, 4))
    
    # Dibujar cada dado
    colores = ['lightcoral', 'lightblue', 'lightgreen']
    for i, (valor, color) in enumerate(zip(dados, colores)):
        dibujar_dado(ax, valor, (i*1.5, 0), color=color)
    
    ax.set_xlim(-0.5, 5)
    ax.set_ylim(-0.5, 1.5)
    ax.set_aspect('equal')
    ax.axis('off')
    
    # Texto con resultado
    suma = dados.sum()
    ax.text(2.25, 1.3, f'Suma: {suma}', fontsize=20, fontweight='bold', 
           ha='center', bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.7))
    
    plt.title('🎲 Tirada de 3 Dados', fontsize=16, fontweight='bold', pad=20)
    plt.tight_layout()
    plt.show()
    
    return dados, suma


# Ejecutar varias veces para ver diferentes tiradas
print("🎲 Simulando tiradas aleatorias...\n")
for i in range(3):
    dados, suma = visualizar_tirada_simple()
    print(f"Tirada {i+1}: Dados = {dados} → Suma = {suma}")

---

## 2.2 Histograma Dinámico (Convergencia Visual)

In [ ]:
def simulacion_progresiva(n_total=10000, pasos=20):
    """
    Muestra cómo el histograma converge a la distribución teórica
    """
    
    fig, ax = plt.subplots(figsize=(12, 6))
    
    # Distribución teórica
    sumas_teoricas = sorted(prob_exactas.keys())
    prob_teoricas = [prob_exactas[s] for s in sumas_teoricas]
    
    # Simular progresivamente
    tamaños = np.logspace(2, np.log10(n_total), pasos, dtype=int)
    
    for idx, n in enumerate(tamaños):
        ax.clear()
        
        # Simular n tiradas
        dados = np.random.randint(1, 7, size=(n, 3))
        sumas = dados.sum(axis=1)
        
        # Histograma normalizado
        conteo = Counter(sumas)
        sumas_vals = sorted(conteo.keys())
        frecuencias = [conteo[s]/n for s in sumas_vals]
        
        # Gráfica
        ax.bar(sumas_vals, frecuencias, alpha=0.7, color='steelblue', 
              label=f'Monte Carlo (N={n})', edgecolor='black')
        ax.plot(sumas_teoricas, prob_teoricas, 'ro-', linewidth=2, 
               markersize=8, label='Distribución teórica')
        
        ax.set_xlabel('Suma', fontsize=12)
        ax.set_ylabel('Probabilidad', fontsize=12)
        ax.set_title(f'Convergencia a la Distribución Teórica (N = {n})', 
                    fontsize=14, fontweight='bold')
        ax.legend(fontsize=11)
        ax.set_ylim(0, 0.16)
        ax.grid(alpha=0.3)
        
        plt.pause(0.5)
    
    plt.show()

# Ejecutar (quita el comentario para ver la animación)
# simulacion_progresiva(n_total=10000, pasos=15)

print("💡 TIP: Descomenta la línea anterior para ver la animación")
print("    Verás cómo el histograma converge a la distribución teórica")

---

# 3️⃣ SECCIÓN 3: Extensiones Creativas

## 3.1 Dados Cargados (Distribuciones No Uniformes)

**Pregunta:** ¿Y si los dados están trucados?

Simulemos un dado que favorece ciertos números (ej: casinos tramposos).

In [ ]:
def dado_cargado(n_tiradas, sesgo_6=0.4):
    """
    Simula un dado trucado que tiene mayor probabilidad de sacar 6
    
    Args:
        n_tiradas: Número de tiradas
        sesgo_6: Probabilidad de sacar 6 (resto se reparte uniformemente)
    
    Returns:
        array: Resultados de las tiradas
    """
    
    # Probabilidades: [1, 2, 3, 4, 5, 6]
    prob_resto = (1 - sesgo_6) / 5
    probabilidades = [prob_resto] * 5 + [sesgo_6]
    
    # Generar tiradas según esta distribución
    resultados = np.random.choice([1, 2, 3, 4, 5, 6], size=n_tiradas, p=probabilidades)
    
    return resultados


def comparar_dados_justos_vs_cargados(n_tiradas=100000):
    """
    Compara distribuciones de suma con dados justos vs trucados
    """
    
    # Dados justos (3 dados)
    dados_justos = np.random.randint(1, 7, size=(n_tiradas, 3))
    sumas_justas = dados_justos.sum(axis=1)
    
    # Dados cargados (todos favorecen el 6 con prob 0.3)
    dados_cargados = np.column_stack([
        dado_cargado(n_tiradas, sesgo_6=0.3),
        dado_cargado(n_tiradas, sesgo_6=0.3),
        dado_cargado(n_tiradas, sesgo_6=0.3)
    ])
    sumas_cargadas = dados_cargados.sum(axis=1)
    
    # Visualización
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    # Histogramas
    ax1.hist(sumas_justas, bins=np.arange(2.5, 19.5, 1), alpha=0.7, 
            density=True, color='steelblue', edgecolor='black', label='Dados justos')
    ax1.hist(sumas_cargadas, bins=np.arange(2.5, 19.5, 1), alpha=0.7, 
            density=True, color='red', edgecolor='black', label='Dados cargados (P(6)=0.3)')
    
    ax1.set_xlabel('Suma', fontsize=12)
    ax1.set_ylabel('Densidad de probabilidad', fontsize=12)
    ax1.set_title('Dados Justos vs Cargados', fontsize=14, fontweight='bold')
    ax1.legend(fontsize=11)
    ax1.grid(alpha=0.3)
    
    # Medias
    media_justa = sumas_justas.mean()
    media_cargada = sumas_cargadas.mean()
    
    categorias = ['Justos', 'Cargados']
    medias = [media_justa, media_cargada]
    colores = ['steelblue', 'red']
    
    ax2.bar(categorias, medias, color=colores, alpha=0.7, edgecolor='black', linewidth=2)
    ax2.axhline(10.5, color='black', linestyle='--', linewidth=2, alpha=0.5, 
               label='Media teórica (justos) = 10.5')
    
    ax2.set_ylabel('Suma media', fontsize=12)
    ax2.set_title('Comparación de Medias', fontsize=14, fontweight='bold')
    ax2.legend(fontsize=11)
    ax2.grid(alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.show()
    
    print("\n📊 RESULTADOS:")
    print(f"  Suma media (dados justos):   {media_justa:.2f}")
    print(f"  Suma media (dados cargados): {media_cargada:.2f}")
    print(f"\n  → Los dados cargados dan sumas ~{media_cargada - media_justa:.1f} puntos más altas")
    print("\n💡 Aplicación: Detectar trampas en casinos usando análisis estadístico")


# Ejecutar comparación
comparar_dados_justos_vs_cargados(n_tiradas=100000)

---

## 3.2 Juego del Craps (Aplicación Real)

**Craps** es un juego de casino muy popular con dados. Reglas simplificadas:

1. **Primera tirada:**
   - Si sacas 7 u 11 → GANAS
   - Si sacas 2, 3 o 12 → PIERDES
   - Cualquier otro número (4, 5, 6, 8, 9, 10) se convierte en tu "punto"

2. **Siguientes tiradas (si tienes "punto"):**
   - Si vuelves a sacar tu punto → GANAS
   - Si sacas 7 → PIERDES
   - Cualquier otro número → Sigues tirando

**Pregunta:** ¿Cuál es la probabilidad de ganar al Craps?

In [ ]:
def jugar_craps():
    """
    Simula una partida completa de Craps
    
    Returns:
        bool: True si gana, False si pierde
    """
    
    # Primera tirada (2 dados)
    primera = np.random.randint(1, 7, size=2).sum()
    
    # Casos inmediatos
    if primera in [7, 11]:
        return True  # Gana automáticamente
    if primera in [2, 3, 12]:
        return False  # Pierde automáticamente
    
    # Establecer "punto"
    punto = primera
    
    # Seguir tirando hasta ganar o perder
    while True:
        tirada = np.random.randint(1, 7, size=2).sum()
        
        if tirada == punto:
            return True  # Sacó su punto → Gana
        if tirada == 7:
            return False  # Sacó 7 → Pierde


def simular_craps(n_partidas=100000):
    """
    Simula muchas partidas de Craps y calcula probabilidad de ganar
    """
    
    resultados = [jugar_craps() for _ in range(n_partidas)]
    prob_ganar = np.mean(resultados)
    
    return prob_ganar, resultados


print("🎰 Simulando 100,000 partidas de Craps...\n")
prob_ganar_craps, _ = simular_craps(n_partidas=100000)

print("="*60)
print("RESULTADOS: JUEGO DEL CRAPS")
print("="*60)
print(f"\nProbabilidad de GANAR: {prob_ganar_craps*100:.3f}%")
print(f"Probabilidad de PERDER: {(1-prob_ganar_craps)*100:.3f}%")
print(f"\nVentaja de la casa: {(1-prob_ganar_craps - prob_ganar_craps)*100:.3f}%")
print("\n💡 Valor teórico: P(ganar) ≈ 49.29% (la casa siempre gana a largo plazo)")
print("\n🎲 Monte Carlo nos permite calcular probabilidades en juegos complejos")
print("   donde la solución analítica es difícil.")

---

## 3.3 Dados Correlacionados (Conexión con Física)

**Idea:** ¿Y si los resultados de los dados están correlacionados?

Esto es análogo a **espines interactuantes** en el modelo de Ising.

**Modelo:** El resultado del dado 2 tiende a imitar al dado 1 (interacción ferromagnética).

In [ ]:
def dados_correlacionados(n_tiradas, correlacion=0.7):
    """
    Simula dados donde el segundo tiende a salir similar al primero
    
    Args:
        correlacion: Probabilidad de que dado2 = dado1
    
    Returns:
        arrays: (dado1, dado2, dado3)
    """
    
    # Dado 1: aleatorio
    dado1 = np.random.randint(1, 7, size=n_tiradas)
    
    # Dado 2: correlacionado con dado1
    dado2 = np.where(
        np.random.rand(n_tiradas) < correlacion,
        dado1,  # Con prob 'correlacion', copia dado1
        np.random.randint(1, 7, size=n_tiradas)  # Sino, aleatorio
    )
    
    # Dado 3: correlacionado con dado2
    dado3 = np.where(
        np.random.rand(n_tiradas) < correlacion,
        dado2,
        np.random.randint(1, 7, size=n_tiradas)
    )
    
    return dado1, dado2, dado3


def comparar_independientes_vs_correlacionados(n_tiradas=100000):
    """
    Compara distribuciones con y sin correlación
    """
    
    # Caso 1: Dados independientes
    dados_indep = np.random.randint(1, 7, size=(n_tiradas, 3))
    sumas_indep = dados_indep.sum(axis=1)
    
    # Caso 2: Dados correlacionados
    d1, d2, d3 = dados_correlacionados(n_tiradas, correlacion=0.7)
    sumas_corr = d1 + d2 + d3
    
    # Visualización
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    # Histogramas
    ax1.hist(sumas_indep, bins=np.arange(2.5, 19.5, 1), alpha=0.7, 
            density=True, color='steelblue', edgecolor='black', label='Independientes')
    ax1.hist(sumas_corr, bins=np.arange(2.5, 19.5, 1), alpha=0.7, 
            density=True, color='orange', edgecolor='black', label='Correlacionados (ρ=0.7)')
    
    ax1.set_xlabel('Suma', fontsize=12)
    ax1.set_ylabel('Densidad de probabilidad', fontsize=12)
    ax1.set_title('Dados Independientes vs Correlacionados', fontsize=14, fontweight='bold')
    ax1.legend(fontsize=11)
    ax1.grid(alpha=0.3)
    
    # Desviación estándar
    std_indep = sumas_indep.std()
    std_corr = sumas_corr.std()
    
    ax2.bar(['Independientes', 'Correlacionados'], [std_indep, std_corr], 
           color=['steelblue', 'orange'], alpha=0.7, edgecolor='black', linewidth=2)
    
    ax2.set_ylabel('Desviación estándar', fontsize=12)
    ax2.set_title('Dispersión de las Sumas', fontsize=14, fontweight='bold')
    ax2.grid(alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.show()
    
    print("\n📊 ANÁLISIS DE CORRELACIÓN:")
    print(f"\nDesviación estándar (independientes):   {std_indep:.3f}")
    print(f"Desviación estándar (correlacionados): {std_corr:.3f}")
    print(f"\n→ La correlación REDUCE la dispersión (dados tienden a salir igual)")
    print("\n🧲 CONEXIÓN CON FÍSICA:")
    print("  - Dados independientes ≈ Sistema paramagnético (espines aleatorios)")
    print("  - Dados correlacionados ≈ Sistema ferromagnético (espines alineados)")
    print("  - ¡Es la misma física del modelo de Ising!")


# Ejecutar comparación
comparar_independientes_vs_correlacionados(n_tiradas=100000)

---

# 4️⃣ SECCIÓN 4: Conceptos Avanzados del PDF

## 4.1 Algoritmo de Metropolis aplicado a Dados

Del PDF: El algoritmo de Metropolis genera configuraciones según el factor de Boltzmann.

**Aplicación creativa:** Usemos Metropolis para muestrear sumas de dados con una distribución objetivo.

In [ ]:
def metropolis_dados(n_pasos=100000, T=1.0, suma_objetivo=10):
    """
    Usa el algoritmo de Metropolis para muestrear configuraciones de dados
    que minimicen la "energía" = |suma - suma_objetivo|
    
    Análogo al modelo de Ising: queremos estados de baja energía
    
    Args:
        T: "Temperatura" (controla cuánto exploramos)
        suma_objetivo: Suma que queremos favorecer
    """
    
    # Estado inicial aleatorio
    dados = np.random.randint(1, 7, size=3)
    
    # Función de energía
    def energia(dados):
        return abs(dados.sum() - suma_objetivo)
    
    # Historia de estados
    historia_sumas = []
    energia_actual = energia(dados)
    
    aceptados = 0
    
    for _ in range(n_pasos):
        # Proponer nuevo estado: cambiar un dado aleatorio
        dados_nuevo = dados.copy()
        idx = np.random.randint(0, 3)
        dados_nuevo[idx] = np.random.randint(1, 7)
        
        # Calcular cambio de energía
        energia_nueva = energia(dados_nuevo)
        delta_E = energia_nueva - energia_actual
        
        # Criterio de Metropolis
        if delta_E < 0:
            # Aceptar (energía más baja)
            dados = dados_nuevo
            energia_actual = energia_nueva
            aceptados += 1
        else:
            # Aceptar con probabilidad exp(-ΔE/T)
            if np.random.rand() < np.exp(-delta_E / T):
                dados = dados_nuevo
                energia_actual = energia_nueva
                aceptados += 1
        
        historia_sumas.append(dados.sum())
    
    tasa_aceptacion = aceptados / n_pasos
    
    return np.array(historia_sumas), tasa_aceptacion


# Ejecutar Metropolis con diferentes temperaturas
print("🔬 Ejecutando algoritmo de Metropolis...\n")

sumas_T_baja, tasa_T_baja = metropolis_dados(n_pasos=50000, T=0.5, suma_objetivo=10)
sumas_T_alta, tasa_T_alta = metropolis_dados(n_pasos=50000, T=5.0, suma_objetivo=10)

# Visualización
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Evolución temporal
ax1.plot(sumas_T_baja[:5000], alpha=0.7, label=f'T=0.5 (baja)', linewidth=1)
ax1.plot(sumas_T_alta[:5000], alpha=0.7, label=f'T=5.0 (alta)', linewidth=1)
ax1.axhline(10, color='red', linestyle='--', linewidth=2, label='Suma objetivo')

ax1.set_xlabel('Paso de Monte Carlo', fontsize=12)
ax1.set_ylabel('Suma de dados', fontsize=12)
ax1.set_title('Evolución del Sistema (primeros 5000 pasos)', fontsize=14, fontweight='bold')
ax1.legend(fontsize=11)
ax1.grid(alpha=0.3)

# Distribuciones finales
ax2.hist(sumas_T_baja, bins=np.arange(2.5, 19.5, 1), alpha=0.7, 
        density=True, color='blue', edgecolor='black', label='T=0.5 (exploración baja)')
ax2.hist(sumas_T_alta, bins=np.arange(2.5, 19.5, 1), alpha=0.7, 
        density=True, color='red', edgecolor='black', label='T=5.0 (exploración alta)')
ax2.axvline(10, color='black', linestyle='--', linewidth=2, label='Objetivo')

ax2.set_xlabel('Suma', fontsize=12)
ax2.set_ylabel('Densidad de probabilidad', fontsize=12)
ax2.set_title('Distribución Muestreada por Metropolis', fontsize=14, fontweight='bold')
ax2.legend(fontsize=11)
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n📊 RESULTADOS:")
print(f"\nT = 0.5 (baja temperatura):")
print(f"  - Tasa de aceptación: {tasa_T_baja*100:.1f}%")
print(f"  - Suma media: {sumas_T_baja.mean():.2f}")
print(f"  - Desviación estándar: {sumas_T_baja.std():.2f}")

print(f"\nT = 5.0 (alta temperatura):")
print(f"  - Tasa de aceptación: {tasa_T_alta*100:.1f}%")
print(f"  - Suma media: {sumas_T_alta.mean():.2f}")
print(f"  - Desviación estándar: {sumas_T_alta.std():.2f}")

print("\n💡 INTERPRETACIÓN (como en Ising):")
print("  - T baja → sistema se queda cerca del objetivo (mínimo local)")
print("  - T alta → sistema explora más, distribución más amplia")
print("  - ¡Es exactamente el mismo principio que en el modelo de Ising del PDF!")

---

# 5️⃣ SECCIÓN 5: Conexión con Física

## 5.1 Random Walk en 2D (Difusión)

**Conexión:** Los dados generan pasos aleatorios → modelo de difusión de partículas.

In [ ]:
def random_walk_2d_con_dados(n_pasos=1000):
    """
    Usa tiradas de dados para generar un random walk en 2D
    
    Regla: 
    - Dado 1-2: mover izquierda
    - Dado 3-4: mover derecha  
    - Dado 5: mover arriba
    - Dado 6: mover abajo
    """
    
    # Posición inicial
    x, y = 0, 0
    
    # Historia de posiciones
    trayectoria_x = [x]
    trayectoria_y = [y]
    
    for _ in range(n_pasos):
        dado = np.random.randint(1, 7)
        
        if dado in [1, 2]:
            x -= 1  # Izquierda
        elif dado in [3, 4]:
            x += 1  # Derecha
        elif dado == 5:
            y += 1  # Arriba
        else:  # dado == 6
            y -= 1  # Abajo
        
        trayectoria_x.append(x)
        trayectoria_y.append(y)
    
    return np.array(trayectoria_x), np.array(trayectoria_y)


# Simular varias trayectorias
n_particulas = 50
n_pasos = 500

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 7))

# Gráfica 1: Trayectorias individuales
for i in range(10):  # Mostrar solo 10 para claridad
    traj_x, traj_y = random_walk_2d_con_dados(n_pasos)
    ax1.plot(traj_x, traj_y, alpha=0.6, linewidth=1.5)
    ax1.plot(traj_x[-1], traj_y[-1], 'o', markersize=8)  # Posición final

ax1.plot(0, 0, 'r*', markersize=20, label='Origen')
ax1.set_xlabel('x', fontsize=12)
ax1.set_ylabel('y', fontsize=12)
ax1.set_title(f'Random Walk 2D (10 trayectorias, {n_pasos} pasos)', 
             fontsize=14, fontweight='bold')
ax1.legend(fontsize=11)
ax1.grid(alpha=0.3)
ax1.axis('equal')

# Gráfica 2: Distribución de posiciones finales
posiciones_finales_x = []
posiciones_finales_y = []

for _ in range(n_particulas):
    traj_x, traj_y = random_walk_2d_con_dados(n_pasos)
    posiciones_finales_x.append(traj_x[-1])
    posiciones_finales_y.append(traj_y[-1])

ax2.scatter(posiciones_finales_x, posiciones_finales_y, s=100, alpha=0.6, 
           edgecolors='black', linewidths=1.5)
ax2.plot(0, 0, 'r*', markersize=20, label='Origen')

# Círculo teórico (difusión)
r_teorico = np.sqrt(n_pasos)  # Distancia RMS teórica
theta = np.linspace(0, 2*np.pi, 100)
ax2.plot(r_teorico * np.cos(theta), r_teorico * np.sin(theta), 'r--', 
        linewidth=2, label=f'Distancia RMS teórica: √{n_pasos} ≈ {r_teorico:.1f}')

ax2.set_xlabel('x', fontsize=12)
ax2.set_ylabel('y', fontsize=12)
ax2.set_title(f'Posiciones Finales ({n_particulas} partículas)', 
             fontsize=14, fontweight='bold')
ax2.legend(fontsize=11)
ax2.grid(alpha=0.3)
ax2.axis('equal')

plt.tight_layout()
plt.show()

# Análisis estadístico
distancias = np.sqrt(np.array(posiciones_finales_x)**2 + np.array(posiciones_finales_y)**2)
distancia_media = distancias.mean()

print("\n📊 ANÁLISIS DE DIFUSIÓN:")
print(f"\nDistancia media desde el origen: {distancia_media:.2f}")
print(f"Distancia RMS teórica: √{n_pasos} = {np.sqrt(n_pasos):.2f}")
print(f"\n→ Concuerda con la teoría de difusión: <r²> ∝ t")
print("\n🧬 APLICACIONES FÍSICAS:")
print("  - Difusión molecular (movimiento browniano)")
print("  - Conducción térmica")
print("  - Propagación de contaminantes")
print("  - ¡Todo modelado con Monte Carlo!")

---

## 5.2 Mapeo Dados ↔ Sistema de Espines (Ising)

**Idea creativa final:** Mapear resultados de dados a configuraciones de espines.

In [ ]:
def dados_a_espines(n_dados=100):
    """
    Convierte tiradas de dados en configuración de espines
    
    Regla: Dados pares (2,4,6) → spin +1 (↑)
           Dados impares (1,3,5) → spin -1 (↓)
    """
    
    dados = np.random.randint(1, 7, size=n_dados)
    espines = np.where(dados % 2 == 0, 1, -1)
    
    return dados, espines


def visualizar_configuracion_espines():
    """
    Visualiza una red de espines generada aleatoriamente con dados
    """
    
    # Red 10x10
    n = 10
    dados, espines = dados_a_espines(n*n)
    espines = espines.reshape(n, n)
    
    # Visualización
    fig, ax = plt.subplots(figsize=(8, 8))
    
    # Mapa de colores: +1 (rojo), -1 (azul)
    im = ax.imshow(espines, cmap='RdBu', vmin=-1, vmax=1, interpolation='nearest')
    
    # Añadir flechas
    for i in range(n):
        for j in range(n):
            if espines[i, j] == 1:
                ax.text(j, i, '↑', ha='center', va='center', fontsize=20, color='white')
            else:
                ax.text(j, i, '↓', ha='center', va='center', fontsize=20, color='white')
    
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_title('Configuración de Espines generada con Dados\n(Pares=↑, Impares=↓)', 
                fontsize=14, fontweight='bold')
    
    # Colorbar
    cbar = plt.colorbar(im, ax=ax, ticks=[-1, 1])
    cbar.ax.set_yticklabels(['↓ (-1)', '↑ (+1)'])
    
    plt.tight_layout()
    plt.show()
    
    # Magnetización
    magnetizacion = espines.sum() / (n*n)
    
    print(f"\n🧲 Magnetización del sistema: M = {magnetizacion:.3f}")
    print(f"   (M=+1: todos ↑, M=-1: todos ↓, M=0: desordenado)")
    print(f"\n→ Con dados justos, M ≈ 0 (sistema paramagnético, T alta)")
    print("\n💡 Conexión con el PDF:")
    print("  - Esta configuración es análoga al modelo de Ising a T > Tc")
    print("  - Para T < Tc necesitaríamos dados correlacionados (Sección 3.3)")


# Ejecutar
visualizar_configuracion_espines()

---

# 📝 CONCLUSIONES Y REFLEXIONES FINALES

## Resumen del Trabajo

En este proyecto hemos explorado el **método de Monte Carlo** aplicado a dados, yendo mucho más allá del ejercicio básico:

### 1. **Ejercicio Base** ✅
- Calculamos analítica y numéricamente las probabilidades pedidas
- Demostramos la convergencia del método (error ∝ 1/√N)

### 2. **Visualizaciones Avanzadas** 🎨
- Animaciones de dados
- Comparaciones teoría vs simulación
- Análisis de convergencia

### 3. **Extensiones Creativas** 🎲
- Dados cargados (detección de trampas)
- Juego del Craps (aplicación real)
- Dados correlacionados (análogo a espines)

### 4. **Conceptos Avanzados** 🔬
- Algoritmo de Metropolis aplicado a dados
- Conexión con el modelo de Ising del PDF
- Efecto de la "temperatura" en el muestreo

### 5. **Conexión con Física** 🧲
- Random walk y difusión
- Mapeo dados ↔ espines
- Sistemas paramagnéticos vs ferromagnéticos

---

## Lecciones Clave de Monte Carlo

1. **Poder del muestreo:** No necesitamos enumerar todos los casos (2^400 en Ising) si muestreamos inteligentemente

2. **Ley de los grandes números:** Con suficientes muestras, las frecuencias convergen a probabilidades

3. **Escalado del error:** σ ∝ 1/√N → para reducir error a la mitad, necesitamos 4x más simulaciones

4. **Algoritmo de Metropolis:** Permite muestrear distribuciones complejas (como e^(-E/kT) en física)

5. **Versatilidad:** Desde dados hasta modelos cuánticos, Monte Carlo es universal

---

## Aplicaciones en Física Estadística

- **Modelo de Ising:** Simulación de transiciones de fase magnéticas
- **Sistemas de muchas partículas:** Gases, líquidos, sólidos
- **Mecánica cuántica:** Path integrals de Feynman
- **Cosmología:** Evolución del universo temprano
- **Física de materiales:** Propiedades a diferentes temperaturas

---

## Código y Reproducibilidad

Todo el código de este notebook es:
- ✅ **Reproducible** (semilla aleatoria fijada)
- ✅ **Modular** (funciones reutilizables)
- ✅ **Documentado** (docstrings y comentarios)
- ✅ **Extensible** (fácil de modificar y ampliar)

---

## Referencias

- **PDF del profesor:** Modelo d'Ising i mètode de Monte Carlo (Cervera & Mafé)
- Metropolis et al. (1953): "Equation of State Calculations by Fast Computing Machines"
- Landau & Binder: "A Guide to Monte Carlo Simulations in Statistical Physics"

---

**Autor:** Carles Ginestar  
**Fecha:** Febrero 2026  
**Asignatura:** Física Estadística